<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/FourSqure_CV%E7%A2%BA%E8%AA%8D0704.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for colab

In [ ]:
import sys
is_colab = False
if "google.colab" in sys.modules:
  is_colab = True

In [ ]:
if is_colab:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if is_colab:
  !mkdir -p ~/.kaggle
  !cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/
  !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
if is_colab:
  !pip install kaggle
  !apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
if is_colab:
  !mkdir input output

In [ ]:
if is_colab:
  !kaggle competitions download -c foursquare-location-matching
  !unzip foursquare-location-matching.zip -d input/foursquare-location-matching

 94% 149M/159M [00:01<00:00, 154MB/s]
100% 159M/159M [00:01<00:00, 124MB/s]
Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [ ]:
if is_colab:
  !kaggle datasets download -d guoyonfan/binary-lgb-baseline
  !unzip /content/binary-lgb-baseline.zip -d input/binary-lgb-baseline

  0% 0.00/3.90M [00:00<?, ?B/s]
100% 3.90M/3.90M [00:00<00:00, 202MB/s]
Archive:  /content/binary-lgb-baseline.zip
  inflating: input/binary-lgb-baseline/improved_lgb_baseline.lgb  
  inflating: input/binary-lgb-baseline/lgb_baseline.lgb  


In [ ]:
if is_colab:
  !cp input/binary-lgb-baseline/*  /content/drive/MyDrive/datas/foursquare

In [ ]:
if is_colab:
  !pip install Levenshtein
  !pip install catboost
  !pip install cdifflib polyleven
else : 
  !cp -r ../input/foursquare-lightgbm-trained/polyleven-0.7/polyleven-0.7 .
  !cp -r ../input/foursquare-lightgbm-trained/cdifflib-1.2.5/cdifflib-1.2.5 .
  !pip install ./polyleven-0.7
  !pip install ./cdifflib-1.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 258 kB 8.4 MB/s 
     |████████████████████████████████| 2.0 MB 43.6 MB/s 
     |████████████████████████████████| 104 kB 48.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 72 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for cdifflib: filename=cdifflib-1.2.5-cp37-cp37m-linux_x86_64.whl size=32988 sha256=1e6713226b4dc372eb3882745ce7f1f1fef317502b49d77946fd7bf922fed23d
  Stored in directory: /root/.cache/pip/wheels/01/b2/78/56b07db93d847924bb3786d6b155675652cbe7f75103bb9c2f
  Created wheel for polyleven: filename=polyleven-0.7-cp37-cp37m-linux_x86_64.whl size=22681 sha256=f21df3f97b3aeed86dfbab795ea7e7ce7b6cfef725d5abc9d26de4495766ea1d
  Stored in directory: /root/.cache/

# change directory

In [ ]:
if is_colab:
  %cd /content/output

/content/output


In [ ]:
if is_colab:
    !pip install pykakasi
    !pip install fuzzywuzzy
else :
    !unzip ../input/foursquare-lightgbm-trained/pykakasi_deps.dontopenthiskaggle/pykakasi_deps.dontopenthiskaggle -d .
    !conda install ./pykakasi_deps/offline_pykakasi.tar.bz2
    !conda install ./pykakasi_deps/offline_jaconv.tar.bz2
    !conda install ./pykakasi_deps/offline_deprecated.tar.bz2
    !pip install ../input/foursquare-lightgbm-trained/fuzzywuzzy-0.18.0-py2.py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 7.6 MB/s 
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15564 sha256=832e74984bf30bd3a45c1be0200ae7e746a360002a2897703321ab5a3cc57e3b
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## restart here to check

In [1]:
import sys
is_colab = False
if "google.colab" in sys.modules:
  is_colab = True

if is_colab:
  %cd /content/output

/content/output


In [2]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import Levenshtein
import difflib
import multiprocessing
import pandas as pd
import numpy as np
import lightgbm as lgb
from collections import Counter
from tqdm.auto import tqdm
from tqdm.notebook import tqdm

tqdm.pandas()

from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from catboost import CatBoostClassifier
import pykakasi

from fuzzywuzzy import fuzz
import math

from collections import defaultdict

from polyleven import levenshtein
from cdifflib import CSequenceMatcher

In [3]:
TRAIN_FEATURES  = [
      # 'kneighbors', 'kneighbors_country','kneighbors_categories', 
       'categories_index', 
       'latdiff',
       'londiff', 'manhattan', 'euclidean', 'haversine', 'name_sim',
       'name_gesh', 'name_leven', 'name_jaro', 'name_lcs', 'name_fuz',
       'name_len_diff', 'name_nleven', 'name_nlcsk', 'name_nlcs',
       'address_sim', 'address_gesh', 'address_leven', 'address_jaro',
       'address_lcs', 'address_fuz', 'address_len_diff', 'address_nleven',
       'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
       'city_lcs', 'city_fuz', 'city_len_diff', 'city_nleven', 'city_nlcsk',
       'city_nlcs', 
       'phone_gesh', 'phone_jaro', 'phone_lcs',
       'categories_sim', 'categories_gesh', 'categories_leven',
       'categories_jaro', 'categories_lcs', 'categories_fuz',
       'categories_len_diff', 'categories_nleven', 'categories_nlcsk',
       'categories_nlcs',       
       'categories',
       'categories2_sim',
       'categories2_gesh', 'categories2_leven', 'categories2_jaro',
       'categories2_lcs', 'categories2_fuz', 'categories2_len_diff',
       'categories2_nleven', 'categories2_nlcsk', 'categories2_nlcs']


In [4]:
## Parameters
num_neighbors = 10
num_neighbors2 = 6
num_neighbors3 = 25
# num_neighbors = 10
# num_neighbors2 = 5
# num_neighbors3 = 22
SEED = 2022
THRESHOLD = 0.5
NUM_SPLIT = 10
# feat_columns = ['dist', 'name', 'address', 'city', 
#             'state', 'zip', 'url', 
#            'phone', 'categories', 'country', 'categories2']
# vec_columns = ['name', 'categories', 'address', 
#                'state', 'url', 'country', 'categories2']

feat_columns = ['dist', 'name', 'address', 'city', 
            'phone', 'categories', 'categories2']
vec_columns = ['name', 'categories', 'address', 'categories2']

#rec_columns = ['name', 'address', 'categories', 'address', 'phone']
rec_columns = ['name', 'address', 'categories2', 'address', 'phone']


def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(SEED)

In [5]:
# 0703 change phone----
sel_cat ={'accessories stores': 0,
 'adult boutiques': 1,
 'advertising agencies': 2,
 'afghan restaurants': 3,
 'african restaurants': 4,
 'airport lounges': 5,
 'airport services': 6,
 'airport terminals': 7,
 'airports': 8,
 'alternative healers': 9,
 'american restaurants': 10,
 'animal shelters': 11,
 'antique shops': 12,
 'aquariums': 13,
 'arcades': 14,
 'argentinian restaurants': 15,
 'art galleries': 16,
 'art museums': 17,
 'arts & crafts stores': 18,
 'asian restaurants': 19,
 'assisted living': 20,
 'athletics & sports': 21,
 'atms': 22,
 'auditoriums': 23,
 'australian restaurants': 24,
 'auto dealerships': 25,
 'auto garages': 26,
 'auto workshops': 27,
 'automotive shops': 28,
 'baby stores': 29,
 'bagel shops': 30,
 'baggage claims': 31,
 'bakeries': 32,
 'ballrooms': 33,
 'banks': 34,
 'bars': 35,
 'baseball fields': 36,
 'baseball stadiums': 37,
 'basketball courts': 38,
 'basketball stadiums': 39,
 'bath houses': 40,
 'bays': 41,
 'bbq joints': 42,
 'beach bars': 43,
 'beaches': 44,
 'bed & breakfasts': 45,
 'beer bars': 46,
 'beer gardens': 47,
 'beer stores': 48,
 'belgian restaurants': 49,
 'big box stores': 50,
 'bike rentals / bike shares': 51,
 'bike shops': 52,
 'bistros': 53,
 'board shops': 54,
 'boarding houses': 55,
 'boats or ferries': 56,
 'bookstores': 57,
 'border crossings': 58,
 'botanical gardens': 59,
 'boutiques': 60,
 'bowling alleys': 61,
 'brazilian restaurants': 62,
 'breakfast spots': 63,
 'breweries': 64,
 'bridal shops': 65,
 'bridges': 66,
 'bubble tea shops': 67,
 'buddhist temples': 68,
 'buffets': 69,
 'buildings': 70,
 'burger joints': 71,
 'burrito places': 72,
 'bus stations': 73,
 'bus stops': 74,
 'business centers': 75,
 'business services': 76,
 'butchers': 77,
 'cafeterias': 78,
 'cafés': 79,
 'cajun / creole restaurants': 80,
 'camera stores': 81,
 'campaign offices': 82,
 'campgrounds': 83,
 'candy stores': 84,
 'cantonese restaurants': 85,
 'capitol buildings': 86,
 'car washes': 87,
 'caribbean restaurants': 88,
 'carpet stores': 89,
 'casinos': 90,
 'castles': 91,
 'cemeteries': 92,
 'cheese shops': 93,
 'chinese restaurants': 94,
 'chiropractors': 95,
 'chocolate shops': 96,
 'churches': 97,
 'cigkofte places': 98,
 'city halls': 99,
 'clothing stores': 100,
 'club houses': 101,
 'cocktail bars': 102,
 'coffee shops': 103,
 'college academic buildings': 104,
 'college administrative buildings': 105,
 'college arts buildings': 106,
 'college auditoriums': 107,
 'college bookstores': 108,
 'college cafeterias': 109,
 'college classrooms': 110,
 'college communications buildings': 111,
 'college engineering buildings': 112,
 'college gyms': 113,
 'college labs': 114,
 'college libraries': 115,
 'college math buildings': 116,
 'college quads': 117,
 'college rec centers': 118,
 'college residence halls': 119,
 'college science buildings': 120,
 'college stadiums': 121,
 'college technology buildings': 122,
 'college theaters': 123,
 'colleges & universities': 124,
 'comedy clubs': 125,
 'comfort food restaurants': 126,
 'community centers': 127,
 'community colleges': 128,
 'concert halls': 129,
 'conference rooms': 130,
 'conferences': 131,
 'construction & landscaping': 132,
 'convenience stores': 133,
 'convention centers': 134,
 'conventions': 135,
 'corporate cafeterias': 136,
 'cosmetics shops': 137,
 'courthouses': 138,
 'coworking spaces': 139,
 'credit unions': 140,
 'creperies': 141,
 'cruise ships': 142,
 'cultural centers': 143,
 'cupcake shops': 144,
 'currency exchanges': 145,
 'dance studios': 146,
 'daycares': 147,
 'delis / bodegas': 148,
 "dentist's offices": 149,
 'department stores': 150,
 'design studios': 151,
 'dessert shops': 152,
 'dim sum restaurants': 153,
 'diners': 154,
 'discount stores': 155,
 'distribution centers': 156,
 'dive bars': 157,
 "doctor's offices": 158,
 'dog runs': 159,
 'donburi restaurants': 160,
 'doner restaurants': 161,
 'donut shops': 162,
 'driving schools': 163,
 'drugstores': 164,
 'dry cleaners': 165,
 'dumpling restaurants': 166,
 'eastern european restaurants': 167,
 'electronics stores': 168,
 'elementary schools': 169,
 'embassies / consulates': 170,
 'emergency rooms': 171,
 'ethiopian restaurants': 172,
 'event services': 173,
 'event spaces': 174,
 'exhibits': 175,
 'eye doctors': 176,
 'fabric shops': 177,
 'factories': 178,
 'fairs': 179,
 'falafel restaurants': 180,
 'farmers markets': 181,
 'farms': 182,
 'fast food restaurants': 183,
 'festivals': 184,
 'fields': 185,
 'filipino restaurants': 186,
 'financial or legal services': 187,
 'fire stations': 188,
 'fish & chips shops': 189,
 'fish markets': 190,
 'flea markets': 191,
 'flower shops': 192,
 'food': 193,
 'food & drink shops': 194,
 'food courts': 195,
 'food services': 196,
 'food stands': 197,
 'food trucks': 198,
 'football stadiums': 199,
 'forests': 200,
 'fountains': 201,
 'frame stores': 202,
 'fraternity houses': 203,
 'french restaurants': 204,
 'fried chicken joints': 205,
 'friteries': 206,
 'frozen yogurt shops': 207,
 'fruit & vegetable stores': 208,
 'funeral homes': 209,
 'furniture / home stores': 210,
 'gaming cafes': 211,
 'garden centers': 212,
 'gardens': 213,
 'gas stations': 214,
 'gastropubs': 215,
 'gay bars': 216,
 'general colleges & universities': 217,
 'general entertainment': 218,
 'general travel': 219,
 'german restaurants': 220,
 'gift shops': 221,
 'golf courses': 222,
 'gourmet shops': 223,
 'government buildings': 224,
 'greek restaurants': 225,
 'grocery stores': 226,
 'gym pools': 227,
 'gymnastics gyms': 228,
 'gyms': 229,
 'gyms or fitness centers': 230,
 'halal restaurants': 231,
 'harbors / marinas': 232,
 'hardware stores': 233,
 'health & beauty services': 234,
 'health food stores': 235,
 'high schools': 236,
 'historic sites': 237,
 'history museums': 238,
 'hobby shops': 239,
 'hockey arenas': 240,
 'home services & repairs': 241,
 'hookah bars': 242,
 'hospital wards': 243,
 'hospitals': 244,
 'hostels': 245,
 'hot dog joints': 246,
 'hot springs': 247,
 'hotel bars': 248,
 'hotels': 249,
 'hotpot restaurants': 250,
 'housing developments': 251,
 'hunting supplies': 252,
 'ice cream shops': 253,
 'indian restaurants': 254,
 'indie movie theaters': 255,
 'indonesian meatball places': 256,
 'indonesian restaurants': 257,
 'industrial estates': 258,
 'insurance offices': 259,
 'internet cafes': 260,
 'it services': 261,
 'italian restaurants': 262,
 'japanese curry restaurants': 263,
 'japanese restaurants': 264,
 'jazz clubs': 265,
 'jewelry stores': 266,
 'juice bars': 267,
 'karaoke bars': 268,
 'kebab restaurants': 269,
 'kids stores': 270,
 'kofte places': 271,
 'korean restaurants': 272,
 'lakes': 273,
 'language schools': 274,
 'latin american restaurants': 275,
 'laundromats': 276,
 'laundry services': 277,
 'law schools': 278,
 'lawyers': 279,
 'leather goods stores': 280,
 'libraries': 281,
 'light rail stations': 282,
 'lighthouses': 283,
 'lighting stores': 284,
 'lingerie stores': 285,
 'liquor stores': 286,
 'locksmiths': 287,
 'lottery retailers': 288,
 'lounges': 289,
 'malay restaurants': 290,
 'markets': 291,
 'martial arts schools': 292,
 'massage studios': 293,
 'mattress stores': 294,
 'medical centers': 295,
 'medical labs': 296,
 'medical schools': 297,
 'medical supply stores': 298,
 'mediterranean restaurants': 299,
 'meeting rooms': 300,
 "men's stores": 301,
 'mental health offices': 302,
 'metro stations': 303,
 'mexican restaurants': 304,
 'middle eastern restaurants': 305,
 'middle schools': 306,
 'military bases': 307,
 'miscellaneous shops': 308,
 'mobile phone shops': 309,
 'monuments / landmarks': 310,
 'mosques': 311,
 'motels': 312,
 'motorcycle shops': 313,
 'mountains': 314,
 'movie theaters': 315,
 'multiplexes': 316,
 'museums': 317,
 'music festivals': 318,
 'music schools': 319,
 'music stores': 320,
 'music venues': 321,
 'nail salons': 322,
 'national parks': 323,
 'nature preserves': 324,
 'new american restaurants': 325,
 'newsstands': 326,
 'night markets': 327,
 'nightclubs': 328,
 'non-profits': 329,
 'noodle houses': 330,
 'nursery schools': 331,
 'offices': 332,
 'optical shops': 333,
 'organic groceries': 334,
 'other events': 335,
 'other great outdoors': 336,
 'other nightlife': 337,
 'other repair shops': 338,
 'outdoor sculptures': 339,
 'outdoor supply stores': 340,
 'outlet stores': 341,
 'palaces': 342,
 'paper / office supplies stores': 343,
 'parking': 344,
 'parks': 345,
 'pastry shops': 346,
 'pedestrian plazas': 347,
 'performing arts venues': 348,
 'perfume shops': 349,
 'peruvian restaurants': 350,
 'pet services': 351,
 'pet stores': 352,
 'pharmacies': 353,
 'photography labs': 354,
 'photography studios': 355,
 'physical therapists': 356,
 'pide places': 357,
 'piers': 358,
 'pizza places': 359,
 'playgrounds': 360,
 'plazas': 361,
 'police stations': 362,
 'pool halls': 363,
 'pools': 364,
 'ports': 365,
 'portuguese restaurants': 366,
 'post offices': 367,
 'preschools': 368,
 'print shops': 369,
 'private schools': 370,
 'professional & other places': 371,
 'public art': 372,
 'pubs': 373,
 'racetracks': 374,
 'radio stations': 375,
 'ramen restaurants': 376,
 'real estate offices': 377,
 'record shops': 378,
 'recreation centers': 379,
 'recruiting agencies': 380,
 'recycling facilities': 381,
 'rental car locations': 382,
 'rental services': 383,
 'residential buildings (apartments / condos)': 384,
 'resorts': 385,
 'rest areas': 386,
 'restaurants': 387,
 'rivers': 388,
 'rock clubs': 389,
 'roof decks': 390,
 'sake bars': 391,
 'salad places': 392,
 'salons / barbershops': 393,
 'sandwich places': 394,
 'scandinavian restaurants': 395,
 'scenic lookouts': 396,
 'schools': 397,
 'science museums': 398,
 'sculpture gardens': 399,
 'seafood restaurants': 400,
 'shipping stores': 401,
 'shoe repair shops': 402,
 'shoe stores': 403,
 'shopping malls': 404,
 'shopping plazas': 405,
 'shops & services': 406,
 'shrines': 407,
 'skate parks': 408,
 'skating rinks': 409,
 'ski areas': 410,
 'smoke shops': 411,
 'smoothie shops': 412,
 'snack places': 413,
 'soba restaurants': 414,
 'soccer fields': 415,
 'soccer stadiums': 416,
 'social clubs': 417,
 'som tum restaurants': 418,
 'sorority houses': 419,
 'soup places': 420,
 'southern / soul food restaurants': 421,
 'souvenir shops': 422,
 'spanish restaurants': 423,
 'spas': 424,
 'speakeasies': 425,
 'spiritual centers': 426,
 'sporting goods shops': 427,
 'sports bars': 428,
 'sports clubs': 429,
 'stables': 430,
 'stadiums': 431,
 'state / provincial parks': 432,
 'stationery stores': 433,
 'steakhouses': 434,
 'storage facilities': 435,
 'street fairs': 436,
 'strip clubs': 437,
 'student centers': 438,
 'supermarkets': 439,
 'supplement shops': 440,
 'surf spots': 441,
 'sushi restaurants': 442,
 'taco places': 443,
 'tailor shops': 444,
 'taiwanese restaurants': 445,
 'tanning salons': 446,
 'tapas restaurants': 447,
 'tattoo parlors': 448,
 'tea rooms': 449,
 'tech startups': 450,
 'temples': 451,
 'tennis courts': 452,
 'thai restaurants': 453,
 'theaters': 454,
 'theme park rides/attractions': 455,
 'theme parks': 456,
 'thrift / vintage stores': 457,
 'toll plazas': 458,
 'tourist information centers': 459,
 'town halls': 460,
 'toy / game stores': 461,
 'tracks': 462,
 'trade schools': 463,
 'trailer parks': 464,
 'trails': 465,
 'train stations': 466,
 'tram stations': 467,
 'transportation services': 468,
 'travel agencies': 469,
 'travel lounges': 470,
 'tunnels': 471,
 'turkish home cooking restaurants': 472,
 'turkish restaurants': 473,
 'tv stations': 474,
 'udon restaurants': 475,
 'universities': 476,
 'urgent care centers': 477,
 'vacation rentals': 478,
 'vegetarian / vegan restaurants': 479,
 'veterinarians': 480,
 'video game stores': 481,
 'video stores': 482,
 'vietnamese restaurants': 483,
 'vineyards': 484,
 'warehouse stores': 485,
 'warehouses': 486,
 'watch shops': 487,
 'water parks': 488,
 'waterfalls': 489,
 'waterfronts': 490,
 'wedding halls': 491,
 'whisky bars': 492,
 'wine bars': 493,
 'wine shops': 494,
 'wineries': 495,
 'wings joints': 496,
 "women's stores": 497,
 'yoga studios': 498,
 'zoos': 499}

In [6]:
sel_cat = {y.lower() : sel_cat[y] for y in sel_cat}

In [7]:
%load_ext Cython

In [8]:
# %%cython
# def LCS(str S, str T):
#     cdef int i, j
#     cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
#     for i in range(len(S)):
#         for j in range(len(T)):
#             dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
#     return dp[len(S)][len(T)]

In [9]:
%%cython
import numpy as np  # noqa
cpdef int LCS(str S, str T):
    cdef int i, j
    cdef int cost
    cdef int v1,v2,v3,v4
    cdef int[:, :] dp = np.zeros((len(S) + 1, len(T) + 1), dtype=np.int32)
    for i in range(len(S)):
        for j in range(len(T)):
            cost = (int)(S[i] == T[j])
            v1 = dp[i, j] + cost
            v2 = dp[i + 1, j]
            v3 = dp[i, j + 1]
            v4 = dp[i + 1, j + 1]
            dp[i + 1, j + 1] = max((v1,v2,v3,v4))
    return dp[len(S)][len(T)]

In [10]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue

        for m in match:
            if base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 


# get manhattan distance
def manhattan(lat1, long1, lat2, long2):
    return np.abs(lat2 - lat1) + np.abs(long2 - long1)

# get haversine distance
def vectorized_haversine(lats1, lats2, longs1, longs2):
    radius = 6371
    dlat=np.radians(lats2 - lats1)
    dlon=np.radians(longs2 - longs1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lats1)) \
        * np.cos(np.radians(lats2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c
    return d

In [11]:
def recall_simple(df):
    threshold = 2
    
    val2id_d = {}
    for col in rec_columns:
        temp_df = df[['id', col]]
        temp_df[col] = temp_df[col].str.lower()
        val2id = temp_df.groupby(col)['id'].apply(set).to_dict()
        val2id_d[col] = val2id
        del val2id
    
    cus_ids = []
    match_ids = []
    for vals in tqdm(df[rec_columns + ['id']].fillna('null').values):
        cus_id = vals[-1]
        match_id = []
        
        rec_match_count = []
        for i in range(len(rec_columns)):
            col = rec_columns[i]
            
            if vals[i] != 'null':
                rec_match_count += list(val2id_d[col][vals[i].lower()])
        rec_match_count = dict(Counter(rec_match_count))
        
        for k, v in rec_match_count.items():
            if v > threshold:
                match_id.append(k)
        
        cus_ids += [cus_id] * len(match_id)
        match_ids += match_id
    
    train_df = pd.DataFrame()
    train_df['id'] = cus_ids
    train_df['match_id'] = match_ids
    train_df = train_df.drop_duplicates()
    del cus_ids, match_ids
    
    num_data = len(train_df)
    num_data_per_id = num_data / train_df['id'].nunique()
    print('Num of data: %s' % num_data)
    print('Num of data per id: %s' % num_data_per_id)
    
    return train_df

In [12]:
# 0703----------
def recall_knn(df, Neighbors = 10, Neighbors2 = 10, Neighbors3 = 10 ):
    print('Start knn grouped by country')
    train_df_country = []
    for country, country_df in tqdm(df.groupby('country')):
        country_df = country_df.reset_index(drop = True)

        neighbors = min(len(country_df), Neighbors)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(country_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                country_df.index)
        dists, nears = knn.kneighbors(
            country_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        # knn.fit(country_df[['latitude','longitude']],
        #         country_df.index)
        # dists, nears = knn.kneighbors(
        #     country_df[['latitude', 'longitude']], 
        #     return_distance = True)

        for k in range(neighbors):            
            cur_df = country_df[['id']]
            cur_df['match_id'] = country_df['id'].values[nears[:, k]]
            # cur_df['kdist_country'] = dists[:, k]
            cur_df['kneighbors_country'] = k
            
            train_df_country.append(cur_df)
    train_df_country = pd.concat(train_df_country)
    
    #-------------------------------------------------
    print('Start knn grouped by categories')
    seld = sel_cat
  
    train_df_categories = []
    cats = df['categories'].fillna("")

    for categories in tqdm(seld.keys()) :
        categories_df = df[cats.apply(lambda x : categories in set(x.split(", ")))]

        if len(categories_df) == 0: continue

        categories_df = categories_df.reset_index(drop = True)

        neighbors = min(len(categories_df), Neighbors2)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(categories_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                categories_df.index)
        dists, nears = knn.kneighbors(
            categories_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)

        for k in range(neighbors):            
            cur_df = categories_df[['id']]
            cur_df['match_id'] = categories_df['id'].values[nears[:, k]]
            # cur_df['kdist_categories'] = dists[:, k]
            cur_df['kneighbors_categories'] = k
            cur_df['categories_index'] = seld[categories]
            
            train_df_categories.append(cur_df)
    train_df_categories = pd.concat(train_df_categories)    
  
    #-------------------------------------------------
    print('Start knn')
    train_df = []
    knn = NearestNeighbors(n_neighbors = Neighbors3)
    knn.fit(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180), df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180))
    # knn.fit(df[['latitude','longitude']], df.index)
    # dists, nears = knn.kneighbors(df[['latitude','longitude']])
    
    for k in range(Neighbors3):            
        cur_df = df[['id']]
        cur_df['match_id'] = df['id'].values[nears[:, k]]
        # cur_df['kdist'] = dists[:, k]
        cur_df['kneighbors'] = k
        train_df.append(cur_df)
    
    train_df = pd.concat(train_df)
    train_df = train_df.merge(train_df_country,
                                 on = ['id', 'match_id'],
                                 how = 'outer')
    #-------------------------------------------------
    train_df = train_df.merge(train_df_categories,
                                on = ['id', 'match_id'],
                                how = 'outer')

    del train_df_country
    
    return train_df

In [13]:
def add_features(df):    
    for col in tqdm(feat_columns):
        if col == 'dist':
            lat = data.loc[df['id']]['latitude'].values
            match_lat = data.loc[df['match_id']]['latitude'].values
            lon = data.loc[df['id']]['longitude'].values
            match_lon = data.loc[df['match_id']]['longitude'].values
            df['latdiff'] = (lat - match_lat)
            df['londiff'] = (lon - match_lon)
            df['manhattan'] = manhattan(lat, lon, match_lat, match_lon)
            df['euclidean'] = (df['latdiff'] ** 2 + df['londiff'] ** 2) ** 0.5
            df['haversine'] = vectorized_haversine(lat, match_lat, lon, match_lon)
            continue
        
        col_values = data.loc[df['id']][col].values.astype(str)
        matcol_values = data.loc[df['match_id']][col].values.astype(str)
        
        if col in vec_columns:
            tv_fit = tfidf_d[col]
            indexs = [id2index_d[i] for i in df['id']]
            match_indexs = [id2index_d[i] for i in df['match_id']]                    
            df[f'{col}_sim'] = tv_fit[indexs].multiply(tv_fit[match_indexs]).\
                                            sum(axis = 1).A.ravel()
        
        geshs = []
        levens = []
        jaros = []
        lcss = []
        fuzwuz = []
        for s, match_s in zip(col_values, matcol_values):
            if s != 'nan' and match_s != 'nan':                    
                geshs.append(CSequenceMatcher(None, s, match_s).ratio())
                levens.append(levenshtein(s, match_s))
                jaros.append(Levenshtein.jaro_winkler(s, match_s))
                lcss.append(LCS(str(s), str(match_s)))
                fuzwuz.append(fuzz.ratio(s, match_s))
            else:
                geshs.append(np.nan)
                levens.append(np.nan)
                jaros.append(np.nan)
                lcss.append(np.nan)
                fuzwuz.append(np.nan)
        
        df[f'{col}_gesh'] = geshs
        df[f'{col}_leven'] = levens
        df[f'{col}_jaro'] = jaros
        df[f'{col}_lcs'] = lcss
        df[f'{col}_fuz'] = fuzwuz
        
        # add 2022.06.26
        # print(col)
        if col == 'categories' : 
          cats = []
          for s, match_s in zip(col_values, matcol_values):
            cats.append(categorical_similarity(s, match_s))
            # print(s,":", match_s, categorical_similarity(s, match_s))
          df[f'{col}'] = cats


        if col not in ['phone', 'zip']:
            df[f'{col}_len'] = list(map(len, col_values))
            df[f'match_{col}_len'] = list(map(len, matcol_values)) 
            df[f'{col}_len_diff'] = np.abs(df[f'{col}_len'] - df[f'match_{col}_len']) /\
                                        df[f'{col}_len'] 
            df[f'{col}_nleven'] = df[f'{col}_leven'] / \
                                    df[[f'{col}_len', f'match_{col}_len']].max(axis = 1)
            
            df[f'{col}_nlcsk'] = df[f'{col}_lcs'] / df[f'match_{col}_len']
            df[f'{col}_nlcs'] = df[f'{col}_lcs'] / df[f'{col}_len']
            
            df = df.drop(f'{col}_len', axis = 1)
            df = df.drop(f'match_{col}_len', axis = 1)
            gc.collect()
            
    return df

## alphabet convert

In [14]:
def convert_japanese_alphabet(df: pd.DataFrame):
    kakasi = pykakasi.kakasi()
    kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
    kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
    kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
    conversion = kakasi.getConverter()

    def convert(row):
        for column in ["name", "address", "city", "state"]:
            try:
                row[column] = conversion.do(row[column])
            except:
                pass
        return row

    df[df["country"] == "JP"] = df[df["country"] == "JP"].progress_apply(convert, axis=1)
    return df

In [15]:
def categorical_similarity(A, B):
    if not A or not B:
        return -1

    A = set(str(A).split(", "))
    B = set(str(B).split(", "))

    # Find intersection of two sets
    nominator = A.intersection(B)

    if len(nominator) >=1 : return 1
    return 0

    # similarity_1 = len(nominator) / len(A)
    # similarity_2 = len(nominator) / len(B)

    # return max(similarity_1, similarity_2)

## for CV

In [16]:
# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
  return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
  return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

# KNNで選ばれたデータの正解のカバー率を計測する
def get_score(input_df: pd.DataFrame):
  scores = []
  for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
    targets = poi2ids[id2poi[id_str]]
    preds = set(matches.split())
    score = len((targets & preds)) / len((targets | preds))
    scores.append(score)
  scores = np.array(scores)
  return scores.mean()

In [17]:
cdict =  {
    'ATMs': 'Banks',
    'ATMs, Banks': 'Banks',
    'Airport Services': 'Airports',
    'Airport Terminals': 'Airports',
    'American Restaurants': 'Fast Food Restaurants',
    'Asian Restaurants': 'Chinese Restaurants',
    'Asian Restaurants, Chinese Restaurants': 'Chinese Restaurants',
    'Auto Dealerships': 'Automotive Shops',
    'Auto Dealerships, Automotive Shops': 'Auto Dealerships',
    'Auto Garages, Auto Workshops, Automotive Shops': 'Automotive Shops',
    'Auto Garages, Automotive Shops': 'Automotive Shops',
    'Auto Workshops, Automotive Shops': 'Automotive Shops',
    'Automotive Shops, Gas Stations': 'Automotive Shops',
    'Bakeries, Cafés': 'Bakeries',
    'Banks, Buildings': 'Banks',
    'Banks, Offices': 'Banks',
    'Beaches, Surf Spots': 'Beaches',
    'Bed & Breakfasts': 'Hotels',
    'Boutiques': 'Clothing Stores',
    'Buildings': 'Offices',
    'Buildings, Offices': 'Offices',
    'Buildings, Residential Buildings (Apartments / Condos)': 'Residential Buildings (Apartments / Condos)',
    'Burger Joints': 'Fast Food Restaurants',
    'Burger Joints, Fast Food Restaurants': 'Fast Food Restaurants',
    'Bus Stations': 'Bus Stops',
    'Business Services, Mobile Phone Shops': 'Mobile Phone Shops',
    'Cafés, Coffee Shops': 'Cafés',
    'Coffee Shops': 'Cafés',
    'College Academic Buildings': 'College Classrooms',
    'College Auditoriums': 'College Classrooms',
    'Convenience Stores, Gas Stations': 'Gas Stations',
    'Coworking Spaces': 'Offices',
    'Coworking Spaces, Offices': 'Offices',
    'Department Stores': 'Shopping Malls',
    'Discount Stores, Supermarkets': 'Grocery Stores',
    "Doctor's Offices": 'Hospitals',
    "Doctor's Offices, Medical Centers": "Doctor's Offices",
    'Drugstores': 'Pharmacies',
    'Drugstores, Pharmacies': 'Pharmacies',
    'Electronics Stores, Mobile Phone Shops': 'Mobile Phone Shops',
    'Fast Food Restaurants, Fried Chicken Joints': 'Fast Food Restaurants',
    'Fast Food Restaurants, Restaurants, Sandwich Places': 'Sandwich Places',
    'Fast Food Restaurants, Sandwich Places': 'Sandwich Places',
    'Fried Chicken Joints': 'Fast Food Restaurants',
    'Furniture / Home Stores, Lighting Stores': 'Furniture / Home Stores',
    'Government Buildings': 'Offices',
    'Grocery Stores': 'Convenience Stores',
    'Grocery Stores, Supermarkets': 'Grocery Stores',
    'Gyms or Fitness Centers': 'Gyms',
    'Gyms, Gyms or Fitness Centers': 'Gyms',
    'Health & Beauty Services, Salons / Barbershops': 'Salons / Barbershops',
    'High Schools': 'College Classrooms',
    'Hospital Wards': 'Hospitals',
    'Hospitals, Medical Centers': 'Hospitals',
    'Hotels': 'Residential Buildings (Apartments / Condos)',
    'Hotels, Resorts': 'Hotels',
    'Housing Developments': 'Residential Buildings (Apartments / Condos)',
    'Housing Developments, Residential Buildings (Apartments / Condos)': 'Residential Buildings (Apartments / Condos)',
    'Indonesian Restaurants': 'Chinese Restaurants',
    'Italian Restaurants': 'Cafés',
    'Italian Restaurants, Pizza Places': 'Italian Restaurants',
    'Japanese Restaurants, Sake Bars': 'Japanese Restaurants',
    'Japanese Restaurants, Sushi Restaurants': 'Japanese Restaurants',
    'Light Rail Stations': 'Metro Stations',
    'Massage Studios, Spas': 'Spas',
    'Medical Centers': "Doctor's Offices",
    'Middle Schools': 'High Schools',
    'Motels': 'Residential Buildings (Apartments / Condos)',
    'Movie Theaters, Multiplexes': 'Movie Theaters',
    'Multiplexes': 'Movie Theaters',
    'Nail Salons, Salons / Barbershops': 'Salons / Barbershops',
    'Noodle Houses': 'Chinese Restaurants',
    'Offices': 'Banks',
    'Other Great Outdoors': 'Parks',
    'Pizza Places, Restaurants': 'Pizza Places',
    'Pubs': 'Bars',
    'Ramen Restaurants': 'Noodle Houses',
    'Resorts': 'Hotels',
    'Restaurants': 'Cafés',
    'Schools': 'High Schools',
    'Shopping Malls': 'Offices',
    'Soccer Stadiums': 'Soccer Fields',
    'Supermarkets': 'Grocery Stores',
    'Sushi Restaurants': 'Japanese Restaurants',
    'Tech Startups': 'Offices',
    'Thai Restaurants': 'Chinese Restaurants',
    'Train Stations': 'Metro Stations',
    'Universities': 'College Classrooms',
    "Women's Stores": 'Clothing Stores'}


In [18]:
cdict = {y.lower() : cdict[y].lower() for y in cdict}

In [19]:
def f(x) :
  if type(x)==float and math.isnan(x) : return x
  if x in cdict : return cdict[x]
  return x

## data load

In [20]:
def get_train(select, pair) :
  data = pd.read_csv('../input/foursquare-location-matching/train.csv')
  tv_ids_d = np.load('/content/drive/MyDrive/datas/foursquare/traindata0703/tv_ids_d.npy', allow_pickle=True).item()
  data = data.set_index('id')
  data = data.loc[tv_ids_d['valid_ids']]
  
  num_sel = int(len(data)*select)
  num_pair = int(num_sel*pair)

  pol_cnt = defaultdict(int)
  for e in data['point_of_interest'] :
    pol_cnt[e] += 1

  selected_poi = set()
  cnt = 0
  for e in pol_cnt :
    if pol_cnt[e] <= 1 : continue
    selected_poi.add(e)
    cnt += pol_cnt[e]
    if cnt >= num_pair : break

  for e in pol_cnt:
    if pol_cnt[e] > 1 : continue
    selected_poi.add(e)
    cnt += pol_cnt[e]
    if cnt >= num_sel : break

  len(selected_poi), cnt, num_sel, num_pair

  selected = []
  for e in data['point_of_interest'] :
    if e in selected_poi :
      selected.append(True)
    else: 
      selected.append(False)

  data = data[selected]
  print("selected = ", len(data))
  return data


In [21]:
from sklearn.model_selection import GroupKFold

## Dada process
if is_colab:
    # data = pd.read_csv('../input/foursquare-location-matching/train.csv')
    # tv_ids_d = np.load('/content/drive/MyDrive/datas/foursquare/traindata0703/tv_ids_d.npy', allow_pickle=True).item()
    # data = data.set_index('id')
    # data = data.loc[tv_ids_d['valid_ids']]
    
    # data = data.iloc[idx] 

    data = get_train(1/20, 0.35) 
    # data = get_train(1/2, 0.35) 
    data = data.reset_index()
    # id --> poi変換用の辞書. １対１の関係
    id2poi = get_id2poi(data)
    # poi --> id変換用の辞書辞書. １対多の関係
    poi2ids = get_poi2ids(data)   
    data = data.drop('point_of_interest', axis = 1)

else :
    data = pd.read_csv('../input/foursquare-location-matching/test.csv')

    if len(data) < 20:
        data = pd.read_csv('../input/foursquare-location-matching/train.csv',
                          nrows = 100)
        data = data.drop('point_of_interest', axis = 1)
   
#need to new dataset
# data = convert_japanese_alphabet(data)


# 0703 change phone----
import re
def phone_mod(x) :
  if type(x)==str:
    x = re.sub(r"[^0-9]","",x)
    if len(x) >= 8:
      return x[-8:]
  return x

data['phone'] = data['phone'].apply(phone_mod)
#----------------------

# sort categories 0626
print("sort categories add 06/26")
cat = []
for e in data['categories'] :
  if type(e) == float :
    cat.append(e)
    continue
  e = e.split(", ")
  e.sort()
  cat.append(", ".join(e))
data['categories'] = cat
del cat
#-------------------
print("add categories2")
data['categories2'] = data['categories'].apply(f)

# special process
def get_lower(x):
    try:
        return x.lower()
    except:
        return x

for col in data.columns:
    if data[col].dtype == object and col != 'id':
        data[col] = data[col].apply(get_lower)
        
id2index_d = dict(zip(data['id'].values, data.index))

tfidf_d = {}
for col in vec_columns:
    tfidf = TfidfVectorizer()
    tv_fit = tfidf.fit_transform(data[col].fillna('nan'))
    tfidf_d[col] = tv_fit

out_df = pd.DataFrame()
out_df['id'] = data['id'].unique().tolist()
out_df['match_id'] = out_df['id']

test_data_simple = recall_simple(data)
test_data = recall_knn(data, num_neighbors, num_neighbors2, num_neighbors3)

print('train data by knn: %s' % len(test_data))
test_data = test_data.merge(test_data_simple,
                             on = ['id', 'match_id'],
                             how = 'outer')
del test_data_simple
gc.collect()

data = data.set_index('id')
print('Num of unique id: %s' % test_data['id'].nunique())
print('Num of test data: %s' % len(test_data))
print(test_data.sample(5))

selected =  28470
sort categories add 06/26
add categories2


  0%|          | 0/28470 [00:00<?, ?it/s]

Num of data: 24072
Num of data per id: 1.3385231316725978
Start knn grouped by country


  0%|          | 0/139 [00:00<?, ?it/s]

Start knn grouped by categories


  0%|          | 0/500 [00:00<?, ?it/s]

Start knn
train data by knn: 848738
Num of unique id: 28470
Num of test data: 852904
                      id          match_id  kneighbors  kneighbors_country  \
68158   E_031b987e8ee91d  E_12d8878b2d8a49         2.0                 2.0   
16368   E_0c54d1b43dd4f3  E_0c54d1b43dd4f3         0.0                 0.0   
517355  E_dedf0355f36e4b  E_020a2658d6df1f        17.0                 NaN   
617477  E_0c9607a746734a  E_161d477e232d07        21.0                 NaN   
522914  E_0227a50074141c  E_1389681a542645        18.0                 NaN   

        kneighbors_categories  categories_index  
68158                     NaN               NaN  
16368                     0.0             332.0  
517355                    NaN               NaN  
617477                    NaN               NaN  
522914                    NaN               NaN  


In [22]:
"""
22
train data by knn: 480978
Num of unique id: 18980
Num of test data: 482329

30
Start knn
train data by knn: 630620
Num of unique id: 18980
Num of test data: 631969 -> 421648

30 
Start knn
train data by knn: 12568899
Num of unique id: 379604
Num of test data: 13064153
"""

'\n22\ntrain data by knn: 480978\nNum of unique id: 18980\nNum of test data: 482329\n\n30\nStart knn\ntrain data by knn: 630620\nNum of unique id: 18980\nNum of test data: 631969 -> 421648\n\n30 \nStart knn\ntrain data by knn: 12568899\nNum of unique id: 379604\nNum of test data: 13064153\n'

## 重複パターンをid, match_id = x, y or y, x を見つけて除外（アンサンブル時はx＝yも消してよいかも

In [23]:
id2idx = {e:i for i, e in enumerate(data.index)}
id0 = [id2idx[e] for e in test_data['id']]
id1 = [id2idx[e] for e in test_data['match_id']]

sel = []
tot, cnt = 0,0
pair = set()
for i in tqdm(range(len(id0))) :
  x, y = id0[i], id1[i]
  # if y == x : 
  #   sel.append(False)
  if (y,x) in pair :
    sel.append(False)
    pair.remove((y, x))
  else :
    sel.append(True)
    pair.add((x, y))
    cnt+=1
  tot+=1

print("Total = {}　remain = {} data shrink rate = {}".format(tot, cnt, cnt/tot))
test_data = test_data[sel].reset_index()

del id2idx, id0, id1
gc.collect()

  0%|          | 0/852904 [00:00<?, ?it/s]

Total = 852904　remain = 571905 data shrink rate = 0.6705385365762149


18

## ---

In [24]:
data.sample(5)

,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,categories2
id,,,,,,,,,,,,
E_1616708066927f,emre apt.,40.211661,28.357988,NaN,NaN,NaN,NaN,tr,NaN,NaN,buildings,buildings
E_a34f223262e192,capt hiram's resort,27.823394,-80.471345,1580 us highway 1,sebastian,fl,32958,us,https://www.hirams.com,25894346,"resorts, seafood restaurants","resorts, seafood restaurants"
E_0918c6e8b661a5,cratos premium hotel - hemingway toplanti salonu,35.332569,33.364454,NaN,NaN,NaN,NaN,cy,NaN,NaN,meeting rooms,meeting rooms
E_0dda9bd12a4138,haliç üniversitesi,41.058940,28.983936,bomonti kampüsü,i̇stanbul,türkiye,NaN,tr,NaN,NaN,universities,universities
E_b3c5fb48a72362,j&y salon,13.808490,100.570794,NaN,NaN,NaN,NaN,th,NaN,NaN,salons / barbershops,salons / barbershops


In [25]:
# add_features(test_data.iloc[:10])

In [28]:
## Model load
mtype = [
    'lightgbm',
    'lightgbm',
    'lightgbm',
    'lightgbm',
    'lightgbm',
    # 'catboost',
    # 'catboost',
    # 'catboost',
    # 'catboost',
    # 'catboost',
]
models = [
  # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0703_fold0.txt',
  # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0703_fold1.txt',
  # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0703_fold2.txt',
  # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0703_fold3.txt',
  # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0703_fold4.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold0.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold1.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold2.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold3.txt',
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm0626_2_fold4.txt',
]
features = [
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           TRAIN_FEATURES,
           ]

mix_model = []
for i, e in enumerate(mtype) :
    if e == 'catboost' :
      m = CatBoostClassifier()
      m.load_model(models[i])
      mix_model.append(m)
    else :
      mix_model.append(lgb.Booster(model_file = models[i]))

mix_model

In [29]:
# x = add_features(test_data.sample(5))

## prediction

In [30]:
%%time
## Prediction
count = 0
start_row = 0
pred_df = pd.DataFrame()
unique_id = test_data['id'].unique().tolist()
num_split_id = len(unique_id) // NUM_SPLIT
for k in range(1, NUM_SPLIT + 1):
    print('Current split: %s' % k)
    end_row = start_row + num_split_id
    if k < NUM_SPLIT:
        cur_id = unique_id[start_row : end_row]
        cur_data = test_data[test_data['id'].isin(cur_id)]
    else:
        cur_id = unique_id[start_row: ]
        cur_data = test_data[test_data['id'].isin(cur_id)]
    
    # add features & model prediction
    cur_data = add_features(cur_data)


    # cur_data[features[0]]=cur_data[features[0]].fillna(-1)
    # print(cur_data.head())
    # cur_data['kdist_diff'] = (cur_data['kdist'] - cur_data['kdist_country']) /\
    #                             cur_data['kdist_country']
    # cur_data['kneighbors_mean'] = cur_data[['kneighbors', 'kneighbors_country']].mean(axis = 1)

    preds = []
    for num, model in enumerate(mix_model) :
      if mtype[num] == 'catboost' :
          res = model.predict_proba(cur_data[features[num]])[:,1]
      else :
          res = model.predict(cur_data[features[num]])#,predict_disable_shape_check=True)
      cur_data['pred{}'.format(num)] = res
      preds.append('pred{}'.format(num))

#    cur_pred_df = cur_data[cur_data['pred'] > THRESHOLD][['id', 'match_id']]
    cur_pred_df = cur_data[['id','match_id']+preds]
    pred_df = pd.concat([pred_df, cur_pred_df])
    
    start_row = end_row
    count += len(cur_data)

    del cur_data, cur_pred_df
    gc.collect()
print(count)

Current split: 1


  0%|          | 0/7 [00:00<?, ?it/s]

Current split: 2


  0%|          | 0/7 [00:00<?, ?it/s]

Current split: 3


...

Current split: 9


  0%|          | 0/7 [00:00<?, ?it/s]

Current split: 10


  0%|          | 0/7 [00:00<?, ?it/s]

571905
CPU times: user 29min 47s, sys: 10.3 s, total: 29min 57s
Wall time: 16min 40s


In [31]:
# Wall time: 4min 49s  catboost
# Wall time: 12min 42s lightgbm

In [32]:
pred_df

,id,match_id,pred0,pred1,pred2,pred3,pred4
0,E_000002eae2a589,E_000002eae2a589,9.999326e-01,9.999822e-01,9.999916e-01,9.999928e-01,9.999630e-01
1,E_000007f24ebc95,E_000007f24ebc95,9.999906e-01,9.999835e-01,9.999986e-01,9.999932e-01,9.999858e-01
2,E_000008a8ba4f48,E_000008a8ba4f48,9.999997e-01,9.999990e-01,9.999996e-01,9.999993e-01,9.999999e-01
3,E_00001d92066153,E_00001d92066153,9.999894e-01,9.999926e-01,9.999975e-01,9.999948e-01,9.999985e-01
4,E_000023d8f4be44,E_000023d8f4be44,9.999941e-01,9.999988e-01,9.999966e-01,9.999983e-01,9.999823e-01
...,...,...,...,...,...,...,...
571900,E_db890502c4db81,E_dd5f1f8f94b6b8,9.922989e-01,9.988594e-01,9.974801e-01,9.948874e-01,9.981756e-01
571901,E_dbd43e20c7eb4d,E_e10ab73bf3930e,5.161185e-01,4.433017e-01,3.602480e-01,6.010706e-01,6.573086e-01
571902,E_dd519eceb68f13,E_e0924919ee5b88,8.732498e-01,9.842825e-01,9.646177e-01,9.834121e-01,9.799510e-01
571903,E_ec561042ec3a79,E_f75fa431505938,1.101003e-01,6.349611e-01,4.168437e-01,2.126644e-02,1.480355e-01


In [33]:
n = len(mix_model)
if n == 5 :
  pred_df['pred']=(pred_df['pred0'] + pred_df['pred1'] + pred_df['pred2'] + pred_df['pred3'] + pred_df['pred4'])/5
elif n == 4 :
  pred_df['pred']=(pred_df['pred0'] + pred_df['pred1'] + pred_df['pred2'] + pred_df['pred3'])/4
elif n == 3 :
  pred_df['pred']=(pred_df['pred0']+pred_df['pred1']+pred_df['pred2'])/3
elif n == 2 :
  pred_df['pred']=(pred_df['pred0']+pred_df['pred1'])/2
else :
  pred_df['pred']=pred_df['pred0']

In [34]:
pred_df.sample(20)

,id,match_id,pred0,pred1,pred2,pred3,pred4,pred
351460,E_015d8e32b559c7,E_150f539f7b96e8,2.064158e-07,5.337752e-07,4.109608e-08,1.093445e-07,7.592101e-08,1.933105e-07
330466,E_f0a3a9ac15db96,E_29c482aa16bb20,3.159317e-02,1.046277e-02,6.172226e-03,2.878055e-02,9.363375e-02,3.412849e-02
436125,E_106455ee6ea1f4,E_0ac6e8db312291,4.185193e-07,5.088424e-06,2.613578e-07,6.640533e-08,2.881892e-07,1.224579e-06
72403,E_025cd6acc28d96,E_0f0412604e42b5,1.848276e-06,1.781635e-05,2.572121e-05,3.027889e-06,1.623530e-06,1.000745e-05
398261,E_100ba401adec71,E_10b259a4cff7c3,6.562764e-07,5.064229e-06,6.687651e-07,2.106925e-06,8.713951e-07,1.873518e-06
369764,E_010f0652e1c9c8,E_056a1804456b8d,9.685457e-08,1.961838e-08,6.608197e-08,1.055952e-09,3.130869e-08,4.298391e-08
36804,E_05491ca6bd5baf,E_5141061019a6b5,1.656064e-04,1.864539e-05,3.637501e-05,2.101584e-04,1.982568e-04,1.258084e-04
263571,E_0a41525ef120be,E_98b243c6f6cca1,1.387185e-05,1.578820e-04,9.562137e-05,7.719748e-05,3.832788e-05,7.658012e-05
452459,E_0bb2afe1b692dd,E_15b8d68f3f24ff,1.719343e-07,6.275815e-07,1.084239e-07,2.305379e-07,1.718751e-06,5.714458e-07
167387,E_05c9a04233dbbc,E_92632a966b1a7b,1.276391e-07,7.976006e-07,1.610078e-06,5.558683e-08,2.889287e-07,5.759667e-07


In [35]:
# if is_colab:
#   cnt = 0
#   tot = len(pred_df)
#   df_sel = pred_df[['id','match_id', 'pred']]
#   for i in range(len(pred_df)) :
#     x, y, p = df_sel.iloc[i]
#     px = id2poi[x]
#     py = id2poi[y]
#     if px == py and p > 0.5 : cnt += 1
#     if px != py and p < 0.5 : cnt += 1
#     if i!=0 and i%10000 == 0:
#       print(cnt, i, cnt/i)
#     # print(x, y, p)
#     # if i == 10:break
#   cnt, tot, cnt/tot

In [36]:
"""
551265 570000 0.9671315789473685 1FOLD 
551136 570000 0.9669052631578947 5FOLD
550725 570000 0.9661842105263158 3FLOD
"""

'\n551265 570000 0.9671315789473685 1FOLD \n551136 570000 0.9669052631578947 5FOLD\n550725 570000 0.9661842105263158 3FLOD\n'

In [37]:
pred_df = pred_df[pred_df['pred'] > THRESHOLD][['id', 'match_id']]
pred_df

,id,match_id
0,E_000002eae2a589,E_000002eae2a589
1,E_000007f24ebc95,E_000007f24ebc95
2,E_000008a8ba4f48,E_000008a8ba4f48
3,E_00001d92066153,E_00001d92066153
4,E_000023d8f4be44,E_000023d8f4be44
...,...,...
571897,E_b8cf618b944115,E_b936c77182f690
571898,E_b9ff997bcef27b,E_cd2a98c8f733fa
571900,E_db890502c4db81,E_dd5f1f8f94b6b8
571901,E_dbd43e20c7eb4d,E_e10ab73bf3930e


## POST PROCESS

In [38]:
## Submission    
out_df = pd.concat([out_df, pred_df])
out_df = out_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
out_df = post_process(out_df)
print('Unique id: %s' % len(out_df))
print(out_df.head())

out_df = out_df[['id', 'matches']]

Unique id: 28470
                 id                                           match_id  \
0  E_000002eae2a589  [E_000002eae2a589, E_000002eae2a589, E_e80db43...   
1  E_000007f24ebc95               [E_000007f24ebc95, E_000007f24ebc95]   
2  E_000008a8ba4f48               [E_000008a8ba4f48, E_000008a8ba4f48]   
3  E_00001d92066153  [E_00001d92066153, E_00001d92066153, E_7e0d8e9...   
4  E_000023d8f4be44  [E_000023d8f4be44, E_000023d8f4be44, E_12453ef...   

                             matches  
0  E_000002eae2a589 E_e80db432029aea  
1                   E_000007f24ebc95  
2                   E_000008a8ba4f48  
3  E_00001d92066153 E_7e0d8e9138dd56  
4  E_000023d8f4be44 E_12453effe251db  


In [39]:
out_df = post_process(out_df)
if is_colab:
  print(f"CV: {get_score(out_df):.6f}")

CV: 0.869521


In [40]:
out_df = post_process(out_df)
if is_colab:
  print(f"CV: {get_score(out_df):.6f}")

CV: 0.869521


In [41]:
out_df[['id', 'matches']].to_csv('submission.csv', index = False)

In [42]:
#0.874401　→　0.874662　→　0.874739　→ 0.867957

# 0.864830 1/20 0.864944(0704model)　CV: 0.865738
# 10万件（約）　0.866533
# 30万件（約）　0.864867

## show CV

In [43]:
if is_colab:
  print(f"CV: {get_score(out_df):.6f}")

CV: 0.869521


In [44]:
""" NEW 06/26
35% CV: 0.875048   ※Trainデータの割合
42% CV: 0.850410
50% CV: 0.816657
35% CV: 0.880970 new 3fold model 6/27 version　Wall time: 7min 16s
35% CV: 0.881284 new 3fold model 6/28 version　Wall time: Wall time: 17min 7s

CV: 0.875414
CV: 0.875414

0.871856 22 , 0.872733 25 , 0.873689 30 , 0.874907 30-3fold 0.874981 30-5fold

catboost: 
0.838987
CV: 0.874766

"""

' NEW 06/26\n35% CV: 0.875048   ※Trainデータの割合\n42% CV: 0.850410\n50% CV: 0.816657\n35% CV: 0.880970 new 3fold model 6/27 version\u3000Wall time: 7min 16s\n35% CV: 0.881284 new 3fold model 6/28 version\u3000Wall time: Wall time: 17min 7s\n\nCV: 0.875414\nCV: 0.875414\n\n0.871856 22 , 0.872733 25 , 0.873689 30 , 0.874907 30-3fold 0.874981 30-5fold\n\ncatboost: \n0.838987\nCV: 0.874766\n\n'

In [45]:
# LB:    0.858 0.864
# 20% -> 0.93x
# 40% -> 0.874
# 45% -> 0.858 0.8583
# 50% -> 0.840
"""
CV: 0.856132
CV: 0.856653 --- 5fold
CV: 0.858016 --- catboost
CV: 0.855997 --- catboost 40000
CV: 0.855418 --- new lightgbm 
CV: 0.857235 --- new lightgbm 5fold
CV: 0.850607 --- new lightgbm 5fold all data
"""

'\nCV: 0.856132\nCV: 0.856653 --- 5fold\nCV: 0.858016 --- catboost\nCV: 0.855997 --- catboost 40000\nCV: 0.855418 --- new lightgbm \nCV: 0.857235 --- new lightgbm 5fold\nCV: 0.850607 --- new lightgbm 5fold all data\n'

In [46]:
# CV: 0.914089
# CV: 0.913176 5fold
# CV: 0.908401 dart model

In [47]:
# CV: 0.914605 no weight no kanji(miss)
#------------------------------------------------------------------
# CV: 0.934487 weighted sqrt(neg/pos) no kanji(miss)
# CV: 0.934492 weighted srqt(neg/pos) 2fold no kanji(miss)
# CV: 0.931351 wdithged sqrt(neg/pos) light version no kanji(miss)
# CV: 0.934244 weighted sqrt(neg/pos) 
# CV: 0.934954 weighted sqrt(neg/pos) middle version kanji 

In [48]:
# CV: 0.907733　LB:0.856

In [49]:
# CV: 0.907162 1fold
# CV: 0.911148 1fold weighted
# CV : 0.909899 2fold (non weight + weighted)
# CV: 0.909890 1fold weight = 2
